<a href="https://colab.research.google.com/github/Szczureczek1983/Repozytorium2/blob/main/Kopia_notatnika_Embeddings_vs_One_Hot_plain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from nltk.corpus import stopwords
import nltk
import numpy as np
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras import Input
from sklearn.preprocessing import LabelEncoder
import collections
import matplotlib.pyplot as plt

# paczki do modelowania
from keras import models
from keras import layers
from keras import regularizers



In [2]:
#Funkcje pomocnicze
def get_closest(x, embeddings, topn=3):
    """
    Get the closest embeddings calculating the euclidean distance
    Parameters
    ----------
    x: np.ndarray
      Vector containing an embedding
    top_k: int, optional
      Get the top k similar embeddings
    Returns
    -------
    dict
      Dict containing the top k similar embeddings to the given x
    """
    # Stack all embeddings in a single matrix. Note: the matrix dimention will be
    # V x D where V is the vocabulary size and D is the embedding dimension
    embedding_matrix = np.array(list(embeddings.values()))
    # Using broadcasting compute distance to each embedding in our vocabulary
    distances = x - embedding_matrix
    # Comoute the magnitude of each distance
    distances = np.linalg.norm(distances, axis=1)
    # Sort distance and keep the smallest k
    min_idx = np.argsort(distances)[:topn]
    return [list(embeddings)[i] for i in min_idx]

In [6]:
#ściagnięcie danych
nltk.download('stopwords')
url = "https://raw.githubusercontent.com/ashutoshmakone/Twitter-US-Airline-Sentiment-classification/main/Dataset/Tweets.csv"
df = pd.read_csv(url, sep=',')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
NB_WORDS = 10000  # Parametr odpowiadający za maksymalną liczbę słów w słowniku (najczęstsze)
NB_START_EPOCHS = 10  # Liczba epok podczas treningu
BATCH_SIZE = 512  # Wielkość mini-batcha
MAX_LEN = 24  # Maksymalna długość sekwencji 
GLOVE_DIM = 300  # Wymiarowość embeddinga GloVe

In [5]:
def deep_model(model, X_train, y_train, X_valid, y_valid):
    '''
    Function to train a multi-class model. The number of epochs and 
    batch_size are set by the constants at the top of the
    notebook. 
    
    Parameters:
        model : model with the chosen architecture
        X_train : training features
        y_train : training target
        X_valid : validation features
        Y_valid : validation target
    Output:
        model training history
    '''
    model.compile(optimizer='rmsprop'
                  , loss='categorical_crossentropy'
                  , metrics=['accuracy'])
    
    history = model.fit(X_train
                       , y_train
                       , epochs=NB_START_EPOCHS
                       , batch_size=BATCH_SIZE
                       , validation_data=(X_valid, y_valid)
                       , verbose=2)
    return history


def eval_metric(history, metric_name):
    '''
    Function to evaluate a trained model on a chosen metric. 
    Training and validation metric are plotted in a
    line chart for each epoch.
    
    Parameters:
        history : model training history
        metric_name : loss or accuracy
    Output:
        line chart with epochs of x-axis and metric on
        y-axis
    '''
    metric = history.history[metric_name]
    val_metric = history.history['val_' + metric_name]

    e = range(1, NB_START_EPOCHS + 1)

    plt.plot(e, metric, 'bo', label='Train ' + metric_name)
    plt.plot(e, val_metric, 'b', label='Validation ' + metric_name)
    plt.legend()
    plt.show()

def test_model(model, X_train, y_train, X_test, y_test, epoch_stop):
    '''
    Function to test the model on new data after training it
    on the full training data with the optimal number of epochs.
    
    Parameters:
        model : trained model
        X_train : training features
        y_train : training target
        X_test : test features
        y_test : test target
        epochs : optimal number of epochs
    Output:
        test accuracy and test loss
    '''
    model.fit(X_train
              , y_train
              , epochs=epoch_stop
              , batch_size=BATCH_SIZE
              , verbose=2)
    results = model.evaluate(X_test, y_test)
    
    return results

def remove_stopwords(input_text):
    '''
    Function to remove English stopwords from a Pandas Series.
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 
    
def remove_mentions(input_text):
    '''
    Function to remove mentions, preceded by @, in a Pandas Series
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    return re.sub(r'@\w+', '', input_text)

In [7]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [8]:
#wybranie z danych tylko kolumny z tekstem i kolumny z sentymentem
df = df.reindex(np.random.permutation(df.index))  
df = df[['text', 'airline_sentiment']]
df.text = df.text.apply(remove_stopwords).apply(remove_mentions)

In [10]:
#PROSZĘ UZUPEŁNIĆ: Podział na próbkę treningową i testową
#wielkość probki testowej - 10%, ziarno dla losowania - 37
X_train, X_test, y_train, y_test = train_test_split(df.text, df.airline_sentiment, test_size = 0.1, random_state = 37 )
print('# Train data samples:', X_train.shape[0])
print('# Test data samples:', X_test.shape[0])
#ograniczenie liczby próbek ze względu na Colaba (jak puszczane lokalnie to nie trzba używać)
IDX = 1200
X_train = X_train[:IDX]
y_train = y_train[:IDX]
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]
print('++++++++++++++++++++++++++++++++++')
print('# Train data samples:', X_train.shape[0])
print('# Test data samples:', X_test.shape[0])

# Train data samples: 13176
# Test data samples: 1464
++++++++++++++++++++++++++++++++++
# Train data samples: 1200
# Test data samples: 1464


In [11]:
tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
# PROSZĘ UZUPEŁNIĆ:wyuczenie powyższego tokenizera na tekście treningowym
tk.fit_on_texts(X_train)

#PROSZĘ UZUPEŁNIĆ:Inferencja wyuczonego tokenizera na tekście treningowym i testowym
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

In [12]:
X_train_seq

[[19, 10, 271, 920, 710, 11, 298, 247, 1398, 1399, 7, 6, 5, 1400],
 [94, 1401],
 [104,
  1402,
  390,
  1403,
  1404,
  337,
  338,
  5,
  30,
  132,
  921,
  133,
  1,
  711,
  922,
  67,
  299,
  248],
 [1, 1405, 119, 35, 300, 95, 467, 120, 68, 2, 55, 923, 924, 1406, 4, 391],
 [1407, 468, 4, 301, 925, 121, 77],
 [31, 567, 120, 1408, 469, 1, 1409, 1410],
 [58, 122, 186, 19, 10, 470, 392, 391],
 [205, 134, 68, 145, 926, 39, 68, 145, 471, 135, 3, 568, 2, 101, 40, 50],
 [222, 1411, 927, 174, 8, 29, 25],
 [272, 78, 31, 1412, 69, 569, 339, 79, 175, 2, 1413, 2, 14, 273],
 [206, 146, 10, 928, 122, 302, 17, 13],
 [570, 393, 187, 174, 571, 188, 249, 105, 223, 50],
 [1414, 712, 30, 713, 90, 929, 28, 85, 340, 13, 102, 930],
 [1415,
  1416,
  1417,
  341,
  14,
  472,
  1418,
  1419,
  714,
  158,
  342,
  2,
  106,
  250,
  14,
  77,
  46],
 [1420, 1421, 107, 1422, 7, 6, 5, 1423],
 [33, 39, 9],
 [20, 394, 337, 931, 572, 303, 146, 715, 3, 55, 224, 932, 64, 74, 175, 120],
 [2, 8, 29, 74, 15, 78, 2

In [13]:
tk.word_index['airline']

42

In [16]:
helper_dict = {v:k for k,v in tk.word_index.items()}
[helper_dict[idx] for idx in X_train_seq[42]]

['chairman',
 'preferred',
 '518',
 '758',
 'lifetime',
 'miles',
 'what',
 'us',
 'air',
 'reward',
 'no',
 'firstclass',
 'upgrades',
 'usual']

In [17]:
X_train.iloc[42]

' Chairman Preferred 518,758 lifetime miles. What US Air reward, no #firstclass upgrades  usual.'

In [18]:
seq_lengths = X_train.apply(lambda x: len(x.split(' ')))
seq_lengths.describe()

count    1200.000000
mean       11.263333
std         4.208771
min         2.000000
25%         8.000000
50%        12.000000
75%        14.000000
max        22.000000
Name: text, dtype: float64

In [ ]:
#PROSZĘ UZUPEŁNIĆ: Padding sekwencji do wartości MAX_LEN (treningowej i testowej)
X_train_seq_trunc = None
X_test_seq_trunc = None

In [ ]:
# X_test_seq_trunc[10]


In [ ]:
#Zastosowanie label encodera na sentymencie i stworzenie wektorów y
le = LabelEncoder()
#PROSZĘ UZUPEŁNIĆ: wytrenowanie na treningowym y Label Encodera oraz infernecja na teście
y_train_le = None
y_test_le = None
#PROSZĘ UZUPEŁNIĆ: stworzenie y kategorycznych - treningowych i testowych
y_train_oh = None
y_test_oh = None

In [ ]:
X_train.shape

## One Hot embeddings

In [ ]:
from sklearn.preprocessing import OneHotEncoder
result_train = []
for seq in X_train_seq_trunc:
    result_train.append(to_categorical(seq, num_classes=NB_WORDS))
    
result_test = []
for seq in X_test_seq_trunc:
    result_test.append(to_categorical(seq, num_classes=NB_WORDS))

In [ ]:
X_train_oh = np.array(result_train)
X_test_oh = np.array(result_test)
X_train_oh.shape

In [ ]:
oh_model = models.Sequential()
#PROSZĘ UZUPEŁNIĆ: dodanie następujących warst do modelu:
# - warstwa wejsciowa o odpowiednim kształcie!
# - warstwa spłaszczająca
# - warstwa gęsta z odpowiednią liczbą neuronów (sprawdź y) i funkcją softmax

oh_model.summary()

In [ ]:
oh_history = deep_model(oh_model, X_train_oh, y_train_oh, X_test_oh, y_test_oh)


In [ ]:
oh_history.history['accuracy'][-1]

In [ ]:
oh_results = test_model(oh_model, X_train_oh, y_train_oh, X_test_oh, y_test_oh, 3)
print('/n')
print('Test accuracy of word glove model: {0:.2f}%'.format(oh_results[1]*100))

In [ ]:
eval_metric(oh_history, 'loss')

In [ ]:
eval_metric(oh_history, 'accuracy')

## Glove embeddings

In [ ]:
# #Glove 6B

!curl -OL http://nlp.stanford.edu/data/glove.6B.zip -o glove.6B.zip
# #wget
# # !wget http://nlp.stanford.edu/data/glove.6B.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   308    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   346    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  6  822M    6 51.6M    0     0  8531k      0  0:01:38  0:00:06  0:01:32 5221k

In [ ]:
# !unzip -o glove.6B.zip
!unzip -o /content/glove.6B.zip

In [ ]:
glove_embeddings = {}
with open('glove.6B.300d.txt') as f:
    glove_embeddings = {l.split()[0]: np.array(l.split()[1:]).astype('float32') for l in f}

In [ ]:
airline_words = ['airplane', 'airline', 'flight', 'luggage', 'djfhaskdjfasdf']
for w in airline_words:
    if w in glove_embeddings.keys():
        print('Found the word {} in the dictionary'.format(w))

In [ ]:
get_closest(glove_embeddings['airplane'], glove_embeddings)

In [ ]:
#stworzenie macierzy mapowania pomiędzy słowami a odpowiadającymi im wektorami
emb_matrix = np.zeros((NB_WORDS, GLOVE_DIM))

for w, i in tk.word_index.items():
    # The word_index contains a token for all words of the training data so we need to limit that
    if i < NB_WORDS:
        vect = glove_embeddings.get(w)
        # Check if the word from the training data occurs in the GloVe word embeddings
        # Otherwise the vector is kept with only zeros
        if vect is not None:
            emb_matrix[i] = vect
    else:
        break

In [ ]:
glove_model = models.Sequential()
#PROSZĘ UZUPEŁNIĆ: dodanie następujących warst do modelu:
# - warstwa embedding o odpowiednim kształcie! (wykorzystaj NB_WORDS, GLOVE_DIM i MAX_LEN)
# - warstwa spłaszczająca
# - warstwa gęsta z odpowiednią liczbą neuronów (sprawdź y) i funkcją softmax

glove_model.summary()

In [ ]:
#UWAGA - musimy zablokować możliwość trenowania warstwy z embeddingami glove!
glove_model.layers[0].set_weights([emb_matrix])
glove_model.layers[0].trainable = False
glove_model.summary()

In [ ]:
glove_history = deep_model(glove_model, X_train_seq_trunc, y_train_oh, X_test_seq_trunc, y_test_oh)
glove_history.history['accuracy'][-1]

In [ ]:
glove_results = test_model(glove_model, X_train_seq_trunc, y_train_oh, X_test_seq_trunc, y_test_oh, 3)
print('/n')
print('Test accuracy of word glove model: {0:.2f}%'.format(glove_results[1]*100))

In [ ]:
eval_metric(glove_history, 'loss')

In [ ]:
eval_metric(glove_history, 'accuracy')

## Extra section: Trainable Embeddings

In [ ]:
emb_model = models.Sequential()
#PROSZĘ UZUPEŁNIĆ: dodanie następujących warst do modelu:
# - warstwa embedding o odpowiednim kształcie - użyj wymiarowości wektora 8 (wykorzystaj NB_WORDS i MAX_LEN)
# - warstwa spłaszczająca
# - warstwa gęsta z odpowiednią liczbą neuronów (sprawdź y) i funkcją softmax

emb_model.summary()

In [ ]:
emb_history = deep_model(emb_model, X_train_seq_trunc, y_train_oh, X_test_seq_trunc, y_test_oh)
emb_history.history['accuracy'][-1]

In [ ]:
emb_results = test_model(emb_model, X_train_seq_trunc, y_train_oh, X_test_seq_trunc, y_test_oh, 3)
print('/n')
print('Test accuracy of word glove model: {0:.2f}%'.format(emb_results[1]*100))

In [ ]:
eval_metric(emb_history, 'loss')

In [ ]:
eval_metric(emb_history, 'accuracy')